## Problem Statement

#### Your client is a Financial Distribution company. Over the last 10 years, they have created an offline distribution channel across country. They sell Financial products to consumers by hiring agents in their network. These agents are freelancers and get commission when they make a product sale.



Overview of your client On-boarding process
The Managers at your client are primarily responsible for recruiting agents. Once a manager has identified a potential applicant, the would explain the business opportunity to the agent. Once the agent provides the consent, an application is made to your client to become an agent. This date is known as application_receipt_date.

In the next 3 months, this potential agent has to undergo a 7 days training at the your client's branch (about Sales processes and various products) and clear a subsequent examination in order to become an agent.




The problem - Who are the best agents?
As is obvious in the above process, there is a significant investment which your cleint makes in identifying, training and recruiting these agents. However, there are a set of agents who do not bring in the expected resultant business.

Your client is looking for help from data scientists like you to help them provide insigths using their past recruitment data. They want to predict the target variable for each potential agent, which would help them identify the right agents to hire.

### Data Ditictionary

In [1]:
#import tensorflow as tf 
import keras
from keras.layers import Dense,Dropout
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

D:\Study\Great Lakes\INtro to python\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Study\Great Lakes\INtro to python\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Study\Great Lakes\INtro to python\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Study\Great Lakes\INtro to python\Anacon

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [4]:
import numpy as np
import pandas as pd 
import os
from sklearn.model_selection import KFold,StratifiedKFold
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight,compute_sample_weight


In [ ]:
path='D:/Study/Hackathon analytics vidhya/Hackathon AV hiring  10092020'
train_df=pd.read_csv(os.path.join(path,'train.csv'))
test_df=pd.read_csv(os.path.join(path,'test.csv'))
submission_df=pd.read_csv(os.path.join(path,'sample.csv'))
combine_set1 =pd.read_csv(os.path.join(path,'combine_data_train_test5'))

In [ ]:
X=combine_set1[combine_set1['Business_Sourced'].isnull()==False].drop(['Business_Sourced'],axis=1)
y=le.fit_transform(combine_set1[combine_set1['Business_Sourced'].isnull()==False]['Business_Sourced'])
y=pd.DataFrame(y,columns=['Business_Sourced'])
X_main_test=combine_set1[combine_set1['Business_Sourced'].isnull()==True].drop(['Business_Sourced'],axis=1)


In [ ]:
y_hat=to_categorical(y)
y_hat=pd.DataFrame(y_hat)
sc_X=StandardScaler()
X=sc_X.fit_transform(X)
X=pd.DataFrame(X)
sc_X_main=StandardScaler()
X_main_test=sc_X_main.fit_transform(X_main_test)
X.head(5)

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X,y_hat,test_size=0.2,random_state=294)

### K fold LGBM

In [ ]:
kf=KFold(n_splits=10,shuffle=True,random_state=2020)
# sc_X=StandardScaler()
# X=pd.DataFrame(sc_X.fit_transform(X))
preds={}
acc_score=0

    
for i,(train_idx,val_idx) in enumerate(kf.split(X)):    

    X_train, y_train = X.iloc[train_idx,:], y_hat.iloc[train_idx]

    X_val, y_val = X.iloc[val_idx, :], y_hat.iloc[val_idx]
    

    print('\nFold: {}\n'.format(i+1))
    #12,0.8,1000
    lg=LGBMClassifier(boosting_type='gbdt',learning_rate=0.08,depth=12,objective='binary',n_estimators=1000,num_class=11,
                     metric='binary_logloss',colsample_bytree=0.5,reg_alpha=2,reg_lambda=2,random_state=294,n_jobs=-1)

#     X_train,y_train=SMOTETomek(random_state=294).fit_resample(X_train,y_train)
    lg.fit(X_train,y_train)

    print(accuracy_score(y_val,lg.predict(X_val)))

    acc_score+=accuracy_score(y_val,lg.predict(X_val))
    
    preds[i+1]=lg.predict(X_main_test)

### ensemble

In [ ]:
#set seed for same results everytime
seed=0
import sklearn.ensemble as ensemble
import sklearn.metrics as metrics
df1 = X.copy()

X=df1.drop('Business_Sourced',1)
y=df1['Business_Sourced']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state =1)

#declare the models
lr = LogisticRegression()
rf=RandomForestClassifier()
adb=ensemble.AdaBoostClassifier()
bgc=ensemble.BaggingClassifier()
gnb = GaussianNB()
knn=KNeighborsClassifier()
dt = DecisionTreeClassifier()
# ab_rf = AdaBoostClassifier(base_estimator=rf,random_state=0)
# ab_dt = AdaBoostClassifier(base_estimator=dt,random_state=0)
# ab_nb=  AdaBoostClassifier(base_estimator=gnb,random_state=0)
# ab_lr=  AdaBoostClassifier(base_estimator=lr,random_state=0)
bgcl_lr = BaggingClassifier(base_estimator=lr, random_state=0)

# ,ab_rf,ab_dt,ab_nb,ab_lr,bgcl_lr

models=[lr,rf,adb,bgc,gnb,knn,dt,bgcl_lr]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ens(X_train,X_test, y_train, y_test):
    for model in models:
            model.fit(X_train, y_train)
            y_test_pred = model.predict(X_test)
            y_test_pred_new=model.predict_proba(X_test)
            y_test_pred_new=y_test_pred_new[:,1]
            train_score=model.score(X_train,y_train)
            test_score=model.score(X_test,y_test)
            p_score=metrics.precision_score(y_test,y_test_pred)
            r_score=metrics.recall_score(y_test,y_test_pred)
            
            ac=metrics.roc_auc_score(y_test,y_test_pred_new)
            
            sctr.append(train_score)
            scte.append(test_score)
            ps.append(p_score)
            rs.append(r_score)
            auc.append(ac)
    return sctr,scte,auc,ps,rs
ens(X_train,X_test, y_train, y_test)
# 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'
ensemble=pd.DataFrame({'names':['Logistic Regression','Random Forest','Ada boost','Bagging',
                                'Naive-Bayes','KNN','Decistion Tree',
                                'bagged LR'],
                       'auc_score':auc,'training':sctr,'testing':scte,'precision':ps,'recall':rs})
ensemble=ensemble.sort_values(by='auc_score',ascending=False).reset_index(drop=True)
ensemble

In [ ]:
## Making predictions:
from sklearn.ensemble import AdaBoostClassifier
adb=AdaBoostClassifier()

adb.fit(X,y)
y_test_pred_adb=adb.predict(dfte1)

finalpred=pd.concat([dfte['ID'],pd.DataFrame(y_test_pred_adb,columns=['Business_Sourced'])],1)
finalpred.to_csv("submission_adbpred2.csv",index=False)

### XGBOOST Classifier


In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [ ]:
xgb=XGBClassifier(learning_rate=0.09,n_estimators=125,max_depth=4,min_child_weight=4,colsample_bytree=0.5,reg_alpha=0.000001 )

In [ ]:
xgb.fit(X,y)
y_test_pred_xgb=xgb.predict(X_main_test)
finalpred=pd.concat([dfte['ID'],pd.DataFrame(y_test_pred_xgb,columns=['Business_Sourced'])],1)
finalpred.to_csv("xgbpred2.csv",index=False)